In [0]:
import os
import keras

os.environ['CUDA_VISIBLE_DEVICES']='0'

In [0]:
train,test=keras.datasets.cifar10.load_data()

In [0]:
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [0]:
x_train, y_train = train
x_test, y_test = test

x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [0]:
from keras.layers import BatchNormalization

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [0]:
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [0]:
import tensorflow as ft
import keras.backend as K

def combined_loss(gamma=2.,alpha=4.):
  gamma=float(gamma)
  alpha=float(alpha)
  
  def focal_loss_fixed(y_true,y_pred):
    epsilon=1e-8
    y_true=tf.convert_to_tensor(y_true,tf.float32)
    y_pred=tf.convert_to_tensor(y_pred,tf.float32)

    model_out=tf.add(y_pred,epsilon)
    ce=tf.multiply(y_true,-tf.log(model_out))
    weight=tf.multiply(y_true,tf.pow(tf.subtract(1.,model_out),gamma))
    fl=tf.multiply(alpha,tf.multiply(weight,ce))
    reduced_fl=tf.reduce_max(fl,axis=1)
    return tf.reduce_mean(reduce_fl)
  return focal_loss_fixed

  ce_weights_list=[0.,0.3,0.5,0.7,1]

In [0]:
import itertools
results={}

for i,ce_w in enumerate(ce_weights_list):
  print('Numbers of exp: %i, ce_weight: %.2f' %(i,ce_w))
  model=build_mlp(input_shape=x_train.shape[1:])
  model.summary()
  optmizer=keras.optimizers.SGD(lr=LEARNING_RATE,nesterov=True,momentum=MOMENTUM)
  model.compile(loss=combined_loss(),metrics=['accuracy'],optimizer=optimizer)
  model.fit(x_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),shuffle=True)

  exp_name_tag=('exp-%s'%(i))
  results[exp_name_tag]={'train-loss':model.history.history['loss'],
                         'valid-loss':model.history.history['val_loss'],
                         'train-acc':model.history.history['acc'],
                         'valide-acc':model.history.history['val_acc']}


In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()